In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### 1.0 Import the required libraries

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_rows',None)

##### A : Non-Engineered Data (Original Data)

In [4]:
interest_rate_non_engineered_df = pd.read_csv('/content/drive/MyDrive/Ijaz - Updated Files/Dissertation - Masters Project/Loan Interest Rate Prediction (ML Model Development)/Interest Rate Model Development - ML/loan_data.csv',low_memory = False)
# display(interest_rate_non_engineered_df.head())
print(interest_rate_non_engineered_df.shape) # (887379, 74)

(887379, 74)


##### B : Engineered Data (Pre-Processed Data)

In [5]:
interest_rate_engineered_df = pd.read_csv('/content/drive/MyDrive/Ijaz - Updated Files/Dissertation - Masters Project/Loan Interest Rate Prediction (ML Model Development)/Interest Rate Model Development - ML/interest_rate_df_engineered.csv')
# display(interest_rate_engineered_df.head())
print(interest_rate_engineered_df.shape) # (757494, 48)

(757494, 48)


##### 1.1 Non-Engineered Data Preparation

In [6]:
print(interest_rate_non_engineered_df.columns)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [7]:
num_cols = interest_rate_non_engineered_df.select_dtypes(include= ['float64','int64']).columns
cat_cols = interest_rate_non_engineered_df.select_dtypes(include= ['object']).columns
print(len(num_cols)) # 51 numerical variables
print(len(cat_cols)) # 23 categorical variables

51
23


In [8]:
# print(interest_rate_non_engineered_df[num_cols].isnull().sum())

##### 1.2 Handling Significant Proportion of Missing Values

##### A : Numerical Variables

In [9]:
numerical_variables_excluded = ['mths_since_last_delinq','mths_since_last_record','mths_since_last_major_derog',
                                'annual_inc_joint','dti_joint','open_acc_6m','open_il_6m','open_il_12m',
                                'open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m',
                                'open_rv_24m','max_bal_bc','all_util','inq_fi','total_cu_tl','inq_last_12m']
print(len(numerical_variables_excluded)) # 19 variables identified that have signficant proportion of missing values.

19


In [10]:
print(interest_rate_non_engineered_df.shape) # (887379, 74)
interest_rate_non_engineered_df = interest_rate_non_engineered_df.drop(columns = numerical_variables_excluded)
print(interest_rate_non_engineered_df.shape) # (887379, 55)

(887379, 74)
(887379, 55)


##### Strategy : Drop the variables from the data frame. The dataset has been reduced from 74 variables to 55 variables.

##### B : Categorical Variables

In [11]:
# print(interest_rate_non_engineered_df[cat_cols].isnull().sum())

In [12]:
categorical_variables_excluded = ['emp_title','desc','verification_status_joint','next_pymnt_d']
print(len(categorical_variables_excluded)) # 4 variables to be excluded with significant proportion of missing values.

4


##### Strategy : Drop the variables from the data frame. The dataset has been reduced from 55 variables to 51 variables.

In [13]:
print(interest_rate_non_engineered_df.shape) # (887379, 55)
interest_rate_non_engineered_df = interest_rate_non_engineered_df.drop(columns = categorical_variables_excluded)
print(interest_rate_non_engineered_df.shape) # (887379, 51)

(887379, 55)
(887379, 51)


##### 1.3 Redundant Variables

In [14]:
redundant_variables = ['id','member_id','url','policy_code']
# The id represents a unqiue identifier given to the loan listing by the lending platform (Lending Club).
# The member id reresents the borrowers loan application id.
# The urls represents the loans listing on the website.
# The policy code represents whether the loan is publicly available to borrowers.

# The id,member id and urls represent unique identifiers and therefore would not provide any additional
# information in the prediction of interest rates.
# On observation of the policy code, the individual loans catered to a policy code of 1 and the joint
# applications catered to a policy code of 2, since we predicting loan interest rates for individual
# loans, this variable would have only a single value and therefore we can exclude from the analysis.

In [15]:
print(interest_rate_non_engineered_df.shape) # (887379, 51)
interest_rate_non_engineered_df = interest_rate_non_engineered_df.drop(columns = redundant_variables)
print(interest_rate_non_engineered_df.shape) # (887379, 47)

(887379, 51)
(887379, 47)


##### Strategy : Drop the variables from the data frame. The dataset has been reduced from 51 variables to 47 variables.

##### 1.4 Subset the Data for Individual Loan Applications

##### The engineered data consisted of 99.94% individual loan applications and the joint applications consisted of 0.06% and this was filtered for individual applications. Therefore for comparing the engineered and non-engineered data, we have to subset the non-engineered data for individual loan applications.

In [16]:
print(interest_rate_non_engineered_df.shape)
interest_rate_non_engineered_df = interest_rate_non_engineered_df[interest_rate_non_engineered_df['application_type'] == 'INDIVIDUAL']
print(interest_rate_non_engineered_df.shape)

(887379, 47)
(886868, 47)


In [17]:
print(interest_rate_non_engineered_df.shape)
interest_rate_non_engineered_df = interest_rate_non_engineered_df.drop(columns = ['application_type'])
print(interest_rate_non_engineered_df.shape)

(886868, 47)
(886868, 46)


In [18]:
# print(interest_rate_non_engineered_df.isnull().sum())

##### 1.5 Remaining Missing Values (Dropped)

In [19]:
print(interest_rate_non_engineered_df.shape)
interest_rate_non_engineered_df = interest_rate_non_engineered_df.dropna()
print(interest_rate_non_engineered_df.shape)

(886868, 46)
(757263, 46)


##### 1.6 Inspect the Final Non-Engineered Data

In [20]:
# print(interest_rate_non_engineered_df.isnull().sum())
display(interest_rate_non_engineered_df.head())

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
42535,27050.0,27050.0,27050.0,36 months,10.99,885.46,B,B2,10+ years,OWN,...,0.0,0.0,Jan-2016,885.46,Dec-2015,0.0,0.0,0.0,114834.0,59900.0
42536,9750.0,9750.0,9750.0,36 months,13.98,333.14,C,C1,1 year,RENT,...,0.0,0.0,Jan-2016,333.14,Jan-2016,0.0,0.0,0.0,14123.0,15100.0
42537,12000.0,12000.0,12000.0,36 months,6.62,368.45,A,A2,10+ years,MORTGAGE,...,0.0,0.0,Jan-2016,368.45,Jan-2016,0.0,0.0,0.0,267646.0,61100.0
42538,12000.0,12000.0,12000.0,36 months,13.53,407.40,B,B5,10+ years,RENT,...,0.0,0.0,Sep-2015,119.17,Jan-2016,0.0,0.0,15386.0,13605.0,8100.0
42539,15000.0,15000.0,15000.0,36 months,8.90,476.30,A,A5,2 years,MORTGAGE,...,0.0,0.0,Jan-2016,476.30,Jan-2016,0.0,0.0,1514.0,272492.0,15400.0


##### 1.7 Copy the [Non-Engineered Data] into a data frame and save it to a CSV file

In [21]:
# interest_rate_non_engineered_df_1 = interest_rate_non_engineered_df.copy()
# interest_rate_non_engineered_df_1.to_csv('interest_rate_non_engineered_df.csv',index = False)